In [1]:
import sys
sys.path.append('..')
import pandas as pd
from adat.models.classification_model import LogisticRegressionOnTfIdf
from sklearn.metrics import roc_auc_score, f1_score

In [2]:
train_data = pd.read_csv('../data/ai_academy_data/train.csv')
test_data = pd.read_csv('../data/ai_academy_data/test.csv')

In [16]:
train_data

,sequences,labels
0,bin3_trans34 bin3_trans34 bin1_trans1 bin1_tra...,1
1,bin3_trans144 bin1_trans3 bin2_trans3 bin0_tra...,3
2,bin1_trans3 bin4_trans1 bin4_trans1 bin2_trans...,1
3,bin1_trans4 bin1_trans3 bin0_trans1 bin1_trans...,2
4,bin2_trans2 bin4_trans1 bin2_trans25 bin1_tran...,3
...,...,...
199995,bin4_trans37 bin2_trans82 bin2_trans24 bin4_tr...,2
199996,bin4_trans11 bin4_trans4 bin1_trans25 bin1_tra...,3
199997,bin2_trans37 bin3_trans106 bin2_trans37 bin3_t...,1
199998,bin3_trans1 bin0_trans9 bin2_trans4 bin0_trans...,1


In [17]:
train_x = train_data.sequences.values
train_y = train_data.labels.values

test_x = test_data.sequences.values
test_y = test_data.labels.values

In [18]:
model = LogisticRegressionOnTfIdf()

In [19]:
model.fit(train_x, train_y)

In [20]:
probs = model.predict(test_x)

In [23]:
import numpy as np

In [24]:
acc = (test_y == probs.argmax(axis=1)).mean()

print(f'Accuracy = {acc}')

Accuracy = 0.4359387207445905


In [26]:
acc = (test_y == np.random.choice(test_y, len(test_y), replace=False)).mean()

print(f'Accuracy random baseline = {acc}')

Accuracy random baseline = 0.25012398518971435


## Naive

In [9]:
import numpy as np

In [10]:
naive_auc = 0.5
print(f'Naive ROC AUC = {naive_auc}')

Naive ROC AUC = 0.5


In [31]:
naive_acc = (test_y == np.zeros(test_y.shape[0])).mean()

print(f'Naive Accuracy = {naive_acc}')

Naive Accuracy = 0.24657087536940792


# Adversarial MCMC sampler vs Random sampler

In [63]:
# path to MCMC output
output = pd.read_csv('../results_2/results.csv')
generated = output.generated_sequence.values
original = output.original.values

output_rand = pd.read_csv('../results_random/results.csv')
generated_rand = output_rand.generated_sequence.values
original_rand = output_rand.original.values

output_hotflip = pd.read_csv('hotflip_output.csv')
generated_hotflip = output_hotflip.generated_sequence.values
original_hotflip = output_hotflip.original.values

In [64]:
output.shape, output_rand.shape, output_hotflip.shape

((14207, 11), (14207, 11), (14207, 4))

In [65]:
def get_results(output, test_y, generated, original):
    num_generated_so_far = generated.shape[0]

    adversarial_probs = model.predict(generated)

    print(f'Average WER = {output.wer.mean()}\nAverage BLEU = {output.bleu.mean()}\n')
    print(f'Naive ROC AUC = {naive_auc}\nNaive Accuracy = {naive_acc}\n')

    non_adversarial_probs = model.predict(original)
    adv_auc = roc_auc_score(y_true=test_y[:num_generated_so_far], y_score=non_adversarial_probs[:, 1])
    print(f'Non-Adversarial ROC AUC = {adv_auc}')
    adv_acc = (test_y[:num_generated_so_far] == non_adversarial_probs.argmax(axis=1)).mean()
    print(f'Non-Adversarial Accuracy = {adv_acc}\n')

    adv_auc = roc_auc_score(y_true=test_y[:num_generated_so_far], y_score=adversarial_probs[:, 1])
    print(f'Adversarial ROC AUC = {adv_auc}')
    adv_acc = (test_y[:num_generated_so_far] == adversarial_probs.argmax(axis=1)).mean()
    print(f'Adversarial Accuracy = {adv_acc}')

In [66]:
print('>>> MCMC\n')
get_results(output, test_y, generated, original)

>>> MCMC

Average WER = 1.8081227563876963
Average BLEU = 0.8201734764653238

Naive ROC AUC = 0.5
Naive Accuracy = 0.554656155416344

Non-Adversarial ROC AUC = 0.7425382836750724
Non-Adversarial Accuracy = 0.6852959808545084

Adversarial ROC AUC = 0.6136349112938747
Adversarial Accuracy = 0.5886534806785387


In [67]:
print('>>> Random\n')
get_results(output_rand, test_y, generated_rand, original_rand)

>>> Random

Average WER = 1.4975012317871472
Average BLEU = 0.8514050227449653

Naive ROC AUC = 0.5
Naive Accuracy = 0.554656155416344

Non-Adversarial ROC AUC = 0.7425382836750724
Non-Adversarial Accuracy = 0.6852959808545084

Adversarial ROC AUC = 0.6678953766750988
Adversarial Accuracy = 0.6265221369747308


In [68]:
print('>>> Hotflip\n')
get_results(output_hotflip, test_y, generated_hotflip, original_hotflip)

>>> Hotflip

Average WER = 6.903357499824031
Average BLEU = 0.2775505447367772

Naive ROC AUC = 0.5
Naive Accuracy = 0.554656155416344

Non-Adversarial ROC AUC = 0.7425382836750724
Non-Adversarial Accuracy = 0.6852959808545084

Adversarial ROC AUC = 0.5762124032929536
Adversarial Accuracy = 0.60660237910889
